# First-Case OR Delay Analysis

This notebook provides a complete analysis of first-case OR delays using sample data. 
Adapt the code to work with your hospital's data.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

In [ ]:
# Load the sample data
df = pd.read_csv('../data/sample_or_delays.csv')

# Convert time columns to datetime
df['scheduled_start'] = pd.to_datetime(df['scheduled_start'], format='%H:%M:%S').dt.time
df['actual_start'] = pd.to_datetime(df['actual_start'], format='%H:%M:%S').dt.time
df['surgery_date'] = pd.to_datetime(df['surgery_date'])

print(f"Dataset contains {len(df)} first-case records")
print(f"Date range: {df['surgery_date'].min()} to {df['surgery_date'].max()}")
df.head()

## 1. Overall Delay Statistics

In [ ]:
# Calculate key metrics
avg_delay = df['delay_minutes'].mean()
median_delay = df['delay_minutes'].median()
on_time_rate = (df['delay_minutes'] <= 5).mean() * 100
severe_delay_rate = (df['delay_minutes'] > 30).mean() * 100

print(f"Average delay: {avg_delay:.1f} minutes")
print(f"Median delay: {median_delay:.1f} minutes")
print(f"On-time start rate (≤5 min): {on_time_rate:.1f}%")
print(f"Severe delay rate (>30 min): {severe_delay_rate:.1f}%")

# Financial impact calculation
total_delay_hours = df['delay_minutes'].sum() / 60
or_cost_per_minute = 100  # $100/minute is typical
financial_impact = df['delay_minutes'].sum() * or_cost_per_minute

print(f"\nTotal delay time: {total_delay_hours:.1f} hours")
print(f"Estimated financial impact: ${financial_impact:,.0f}")

## 2. Delay Distribution Analysis

In [ ]:
# Create delay distribution plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(df['delay_minutes'], bins=20, edgecolor='black', alpha=0.7)
ax1.axvline(avg_delay, color='red', linestyle='--', label=f'Average: {avg_delay:.1f} min')
ax1.axvline(5, color='green', linestyle='--', label='On-time threshold')
ax1.set_xlabel('Delay Minutes')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of First-Case Delays')
ax1.legend()

# Box plot by OR room
df.boxplot(column='delay_minutes', by='or_room', ax=ax2)
ax2.set_xlabel('OR Room')
ax2.set_ylabel('Delay Minutes')
ax2.set_title('Delay Distribution by OR Room')

plt.tight_layout()
plt.show()

## 3. Root Cause Analysis

In [ ]:
# Analyze delay reasons
delay_reasons = df[df['delay_minutes'] > 5]['delay_reason'].value_counts()
delay_reason_pct = delay_reasons / delay_reasons.sum() * 100

# Create root cause visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
ax1.pie(delay_reason_pct, labels=delay_reason_pct.index, autopct='%1.1f%%')
ax1.set_title('Root Causes of Delays (>5 minutes)')

# Average delay by reason
avg_delay_by_reason = df[df['delay_minutes'] > 5].groupby('delay_reason')['delay_minutes'].mean().sort_values(ascending=False)
avg_delay_by_reason.plot(kind='barh', ax=ax2)
ax2.set_xlabel('Average Delay (minutes)')
ax2.set_title('Average Delay by Root Cause')

plt.tight_layout()
plt.show()

print("\nDelay Reason Analysis:")
for reason, pct in delay_reason_pct.items():
    avg = avg_delay_by_reason.get(reason, 0)
    print(f"{reason}: {pct:.1f}% of delays, averaging {avg:.1f} minutes")

## 4. Day of Week Pattern Analysis

In [ ]:
# Analyze patterns by day of week
day_analysis = df.groupby('day_of_week').agg({
    'delay_minutes': ['mean', 'median', 'std', 'count'],
    'case_id': 'count'
}).round(1)

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
day_analysis = day_analysis.reindex(day_order)

# Plot day of week patterns
fig, ax = plt.subplots(figsize=(10, 6))
x = range(len(day_order))
ax.bar(x, day_analysis['delay_minutes']['mean'], yerr=day_analysis['delay_minutes']['std'], 
       capsize=5, alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(day_order)
ax.set_ylabel('Average Delay (minutes)')
ax.set_title('First-Case Delays by Day of Week')
ax.axhline(y=avg_delay, color='red', linestyle='--', label=f'Overall Average: {avg_delay:.1f} min')
ax.legend()

plt.tight_layout()
plt.show()

print("\nDay of Week Analysis:")
print(day_analysis['delay_minutes'])

## 5. OR Room Performance Comparison

In [ ]:
# Compare OR room performance
room_performance = df.groupby('or_room').agg({
    'delay_minutes': ['mean', 'count'],
    'case_id': lambda x: (df[df['or_room'] == x.iloc[0]]['delay_minutes'] <= 5).sum()
}).round(1)

room_performance.columns = ['avg_delay', 'total_cases', 'on_time_cases']
room_performance['on_time_rate'] = (room_performance['on_time_cases'] / room_performance['total_cases'] * 100).round(1)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Average delay by room
room_performance['avg_delay'].plot(kind='bar', ax=ax1)
ax1.set_ylabel('Average Delay (minutes)')
ax1.set_title('Average Delay by OR Room')
ax1.axhline(y=avg_delay, color='red', linestyle='--')

# On-time rate by room
room_performance['on_time_rate'].plot(kind='bar', ax=ax2, color='green')
ax2.set_ylabel('On-Time Start Rate (%)')
ax2.set_title('On-Time Performance by OR Room')
ax2.axhline(y=80, color='red', linestyle='--', label='Target: 80%')
ax2.legend()

plt.tight_layout()
plt.show()

print("\nOR Room Performance Summary:")
print(room_performance)

## 6. Financial Impact Calculator

In [ ]:
# Calculate detailed financial impact
def calculate_financial_impact(df, or_cost_per_min=100, overtime_threshold_min=30, overtime_rate=1.5):
    # Direct OR time cost
    total_delay_cost = df['delay_minutes'].sum() * or_cost_per_min
    
    # Overtime costs (delays > 30 min often cause overtime)
    overtime_delays = df[df['delay_minutes'] > overtime_threshold_min]
    overtime_minutes = (overtime_delays['delay_minutes'] - overtime_threshold_min).sum()
    overtime_cost = overtime_minutes * or_cost_per_min * (overtime_rate - 1)
    
    # Opportunity cost (lost cases)
    avg_case_duration = df['case_duration_min'].mean()
    potential_additional_cases = total_delay_cost / or_cost_per_min / avg_case_duration
    avg_case_revenue = 5000  # Conservative estimate
    opportunity_cost = potential_additional_cases * avg_case_revenue
    
    return {
        'total_delay_minutes': df['delay_minutes'].sum(),
        'direct_cost': total_delay_cost,
        'overtime_cost': overtime_cost,
        'opportunity_cost': opportunity_cost,
        'total_impact': total_delay_cost + overtime_cost + opportunity_cost
    }

# Calculate annual projection
sample_days = (df['surgery_date'].max() - df['surgery_date'].min()).days + 1
annual_multiplier = 250 / sample_days  # 250 operating days per year

sample_impact = calculate_financial_impact(df)
annual_impact = {k: v * annual_multiplier for k, v in sample_impact.items()}

print("Financial Impact Analysis")
print("=" * 40)
print(f"Sample Period ({sample_days} days):")
for key, value in sample_impact.items():
    if 'cost' in key or 'impact' in key:
        print(f"  {key.replace('_', ' ').title()}: ${value:,.0f}")
    else:
        print(f"  {key.replace('_', ' ').title()}: {value:,.0f}")

print(f"\nAnnual Projection (250 operating days):")
for key, value in annual_impact.items():
    if 'cost' in key or 'impact' in key:
        print(f"  {key.replace('_', ' ').title()}: ${value:,.0f}")
    else:
        print(f"  {key.replace('_', ' ').title()}: {value:,.0f}")

# ROI calculation for interventions
intervention_cost = 50000  # One-time implementation cost
expected_reduction = 0.6  # 60% reduction based on pilot data
annual_savings = annual_impact['total_impact'] * expected_reduction
roi = (annual_savings - intervention_cost) / intervention_cost * 100
payback_months = intervention_cost / (annual_savings / 12)

print(f"\nROI Analysis (60% reduction scenario):")
print(f"  Implementation cost: ${intervention_cost:,.0f}")
print(f"  Expected annual savings: ${annual_savings:,.0f}")
print(f"  First-year ROI: {roi:.0f}%")
print(f"  Payback period: {payback_months:.1f} months")

## 7. Actionable Insights & Recommendations

In [ ]:
# Generate targeted recommendations based on analysis
print("PRIORITY RECOMMENDATIONS")
print("=" * 50)

# 1. Equipment readiness (top cause)
equipment_delays = df[df['delay_reason'] == 'Equipment Not Ready']
if len(equipment_delays) > 0:
    equipment_impact = equipment_delays['delay_minutes'].sum() / df['delay_minutes'].sum() * 100
    print(f"\n1. EQUIPMENT READINESS ({equipment_impact:.0f}% of total delays)")
    print("   - Implement evening equipment checklist")
    print("   - Assign OR tech for PM equipment verification")
    print(f"   - Potential savings: ${annual_impact['total_impact'] * equipment_impact/100:,.0f}/year")

# 2. Day of week patterns
monday_delay = day_analysis.loc['Monday', ('delay_minutes', 'mean')]
if monday_delay > avg_delay * 1.3:
    print(f"\n2. MONDAY SYNDROME (delays {monday_delay/avg_delay:.0f}x average)")
    print("   - Sunday evening equipment prep team")
    print("   - Monday-specific morning huddle at 6:45 AM")
    print("   - Weekend case cart preparation")

# 3. Room-specific issues
worst_room = room_performance['avg_delay'].idxmax()
if room_performance.loc[worst_room, 'avg_delay'] > avg_delay * 1.5:
    print(f"\n3. TARGETED INTERVENTION FOR {worst_room}")
    print(f"   - Average delay: {room_performance.loc[worst_room, 'avg_delay']:.0f} min")
    print("   - Dedicated first-case team assignment")
    print("   - Equipment standardization review")

# 4. Quick wins
print("\n4. QUICK WINS (implement this week)")
print("   - Post yesterday's on-time performance in each OR")
print("   - Text reminder to first-case teams at 6:30 AM")
print("   - Celebrate on-time starts with team recognition")

print("\n" + "=" * 50)
print("Expected combined impact: 60-80% reduction in delays")
print(f"Projected annual savings: ${annual_savings:,.0f}")
print(f"Time to break even: {payback_months:.0f} months")